In [2]:
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import callbacks
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

warnings.filterwarnings(action='ignore')
tf.config.set_visible_devices([], 'GPU')

In [3]:
df = pd.read_csv("data/modified_train_0417.csv")
x_train = df.drop(columns=['loan_status', 'addr_state', 'loan_amnt_intallment_rate'])
y_train = df['loan_status']

In [4]:
nan_indices = np.isnan(x_train).any(axis=1)
x_train = x_train[~nan_indices]
y_train = y_train[~nan_indices]

In [5]:
# minmax scaler 찾아보기
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_train)

# PCA로 차원 축소
pca = PCA(n_components=90)
X_pca = pca.fit_transform(X_scaled)
x_train = X_pca.astype('float32')
# stratify로 학습에 용이하게 비율 유지
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

In [6]:
y_train=to_categorical(y_train, 2).astype(int)
y_validation=to_categorical(y_validation, 2).astype(int)
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# y_train = y_train.values
# y_train = encoder.fit_transform(y_train.reshape(-1, 1))

In [7]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [9]:
def build_model(optimizer):
    # Define and compile the model
    model = keras.Sequential([
        Dense(45, input_dim=90, activation='relu'),
        Dropout(0.2),
        Dense(45, activation='relu'),
        Dropout(0.2),
        Dense(45, activation='relu'),
        Dropout(0.2),
        Dense(10),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model
kfold = KFold(random_state=30,
           n_splits=5,
           shuffle=True
          )
model = KerasClassifier(build_fn = build_model)
parameters = {'batch_size': [1000,
                             5000,
                             10000],
              'epochs': [20,
                         45,
                         60,
                         100],
              'optimizer': ['adam', 
                            'SGD'
                            ]}
random_search = RandomizedSearchCV(estimator = model,
                           param_distributions= parameters,
                           cv = kfold)
# early_stopping = EarlyStopping(monitor='val_loss',min_delta=0.01)
random_search.fit(x_train, y_train)
print(f"Best params: {random_search.best_params_}")
print(f"Best average accuracy: {random_search.best_score_}")

Epoch 1/100


2024-04-17 21:17:32.414999: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


140/140 [==============================] - 1s 5ms/step - loss: 0.5153 - accuracy: 0.7832
Epoch 2/100
140/140 [==============================] - 1s 5ms/step - loss: 0.4733 - accuracy: 0.7956
Epoch 3/100
140/140 [==============================] - 1s 5ms/step - loss: 0.4670 - accuracy: 0.7964
Epoch 4/100
140/140 [==============================] - 1s 5ms/step - loss: 0.4638 - accuracy: 0.7971
Epoch 5/100
140/140 [==============================] - 1s 5ms/step - loss: 0.4621 - accuracy: 0.7974
Epoch 6/100
140/140 [==============================] - 1s 5ms/step - loss: 0.4609 - accuracy: 0.7979
Epoch 7/100
140/140 [==============================] - 1s 5ms/step - loss: 0.4599 - accuracy: 0.7983
Epoch 8/100
140/140 [==============================] - 1s 5ms/step - loss: 0.4594 - accuracy: 0.7982
Epoch 9/100
140/140 [==============================] - 1s 5ms/step - loss: 0.4590 - accuracy: 0.7981
Epoch 10/100
140/140 [==============================] - 1s 5ms/step - loss: 0.4586 - accuracy: 0.7984
E

아래는 layer 관련 코드

In [19]:
def build_model(num_layers, num_nodes):
    # Define and compile the model
    model = keras.Sequential()
    model.add(Dense(num_nodes, input_dim=90, activation='relu'))
    for _ in range(num_layers):
        model.add(Dense(num_nodes, activation='relu'))
        model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy"])
    return model
kfold = KFold(random_state=30,
           n_splits=2,
           shuffle=True
          )
model = KerasClassifier(build_fn = build_model, epochs=10, batch_size=100, verbose=1)
parameters = {'num_layers': [2, 3],
              'num_nodes': [30, 45, 60]
              }
random_search = RandomizedSearchCV(estimator= model,
                           param_distributions= parameters,
                           cv = kfold)
random_search.fit(x_train, y_train)
print(f"Best params: {random_search.best_params_}")
print(f"Best average accuracy: {random_search.best_score_}")

Epoch 1/10
4368/4368 [==============================] - 4s 802us/step - loss: 0.4662 - accuracy: 0.7975
Epoch 2/10
4368/4368 [==============================] - 3s 796us/step - loss: 0.4586 - accuracy: 0.7986
Epoch 3/10
4368/4368 [==============================] - 3s 789us/step - loss: 0.4572 - accuracy: 0.7993
Epoch 4/10
4368/4368 [==============================] - 4s 802us/step - loss: 0.4561 - accuracy: 0.7995
Epoch 5/10
4368/4368 [==============================] - 3s 776us/step - loss: 0.4552 - accuracy: 0.7998
Epoch 6/10
4368/4368 [==============================] - 3s 774us/step - loss: 0.4547 - accuracy: 0.8001
Epoch 7/10
4368/4368 [==============================] - 3s 784us/step - loss: 0.4539 - accuracy: 0.8003
Epoch 8/10
4368/4368 [==============================] - 3s 774us/step - loss: 0.4535 - accuracy: 0.8003
Epoch 9/10
4368/4368 [==============================] - 3s 791us/step - loss: 0.4531 - accuracy: 0.8004
Epoch 10/10
4368/4368 [==============================] - 2s 398u

In [26]:
best_params_with_layer = random_search.best_params_
valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []
# layers
model = keras.Sequential()
model.add(Dense(best_params_with_layer['num_nodes'][0], input_dim=90, activation='relu'))
for _ in range(best_params_with_layer['num_layers'][0]):
    model.add(Dense(best_params_with_layer['num_nodes'][0], activation='relu'))
    model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy", f1_score, recall, precision])
skf = KFold(n_splits=5, shuffle=True, random_state=30)
for train_index, val_index in skf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    Y_train_fold, Y_val_fold = y_train[train_index], y_train[val_index]
    # Train the model
    model.fit(X_train_fold, Y_train_fold, 
              batch_size=10000, 
              epochs=100, 
              verbose=1, 
              #callbacks=[EarlyStopping(min_delta=0.001, monitor="loss")]
              )
    # Evaluate the model on validation data
    valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_val_fold, Y_val_fold)
    valid_accs.append(valid_acc)
    valid_f1s.append(valid_f1)
    valid_recalls.append(valid_recall)
    valid_precisions.append(valid_precision)
    print("===================================")
    print("Validation accuracy:", valid_acc)
    print("Validation F1-score:", valid_f1)
    print("Validation recall:", valid_recall)
    print("Validation precision:", valid_precision)
print("###################################")
print("avg Validation accuracy:", np.mean(valid_accs))
print("avg Validation F1-score:", np.mean(valid_f1s))
print("avg Validation recall:", np.mean(valid_recalls))
print("avg Validation precision:", np.mean(valid_precisions))


Epoch 1/100
70/70 [==============================] - 1s 7ms/step - loss: 0.5151 - accuracy: 0.7864 - f1_score: 0.7864 - recall: 0.7864 - precision: 0.7864
Epoch 2/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4726 - accuracy: 0.7959 - f1_score: 0.7959 - recall: 0.7959 - precision: 0.7959
Epoch 3/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4662 - accuracy: 0.7969 - f1_score: 0.7969 - recall: 0.7969 - precision: 0.7969
Epoch 4/100
70/70 [==============================] - 1s 8ms/step - loss: 0.4634 - accuracy: 0.7975 - f1_score: 0.7975 - recall: 0.7975 - precision: 0.7975
Epoch 5/100
70/70 [==============================] - 1s 8ms/step - loss: 0.4611 - accuracy: 0.7980 - f1_score: 0.7980 - recall: 0.7980 - precision: 0.7980
Epoch 6/100
70/70 [==============================] - 1s 8ms/step - loss: 0.4598 - accuracy: 0.7983 - f1_score: 0.7982 - recall: 0.7982 - precision: 0.7982
Epoch 7/100
70/70 [==============================] - 1s 8ms/step - los

In [27]:
best_params_with_layer = random_search.best_params_
valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []
# layers
model = keras.Sequential()
model.add(Dense(best_params_with_layer['num_nodes'][0], input_dim=90, activation='relu'))
for _ in range(best_params_with_layer['num_layers'][0]):
    model.add(Dense(best_params_with_layer['num_nodes'][0], activation='relu'))
    model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy", f1_score, recall, precision])
skf = KFold(n_splits=5, shuffle=True, random_state=30)
for train_index, val_index in skf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    Y_train_fold, Y_val_fold = y_train[train_index], y_train[val_index]
    # Train the model
    model.fit(X_train_fold, Y_train_fold, 
              batch_size=10000, 
              epochs=100, 
              verbose=1, 
              callbacks=[EarlyStopping(min_delta=0.001, monitor="loss")]
              )
    # Evaluate the model on validation data
    valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_val_fold, Y_val_fold)
    valid_accs.append(valid_acc)
    valid_f1s.append(valid_f1)
    valid_recalls.append(valid_recall)
    valid_precisions.append(valid_precision)
    print("===================================")
    print("Validation accuracy:", valid_acc)
    print("Validation F1-score:", valid_f1)
    print("Validation recall:", valid_recall)
    print("Validation precision:", valid_precision)
print("###################################")
print("avg Validation accuracy:", np.mean(valid_accs))
print("avg Validation F1-score:", np.mean(valid_f1s))
print("avg Validation recall:", np.mean(valid_recalls))
print("avg Validation precision:", np.mean(valid_precisions))

Epoch 1/100
70/70 [==============================] - 1s 7ms/step - loss: 0.4988 - accuracy: 0.7951 - f1_score: 0.7951 - recall: 0.7951 - precision: 0.7951
Epoch 2/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4677 - accuracy: 0.7965 - f1_score: 0.7965 - recall: 0.7965 - precision: 0.7965
Epoch 3/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4632 - accuracy: 0.7973 - f1_score: 0.7973 - recall: 0.7973 - precision: 0.7973
Epoch 4/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4607 - accuracy: 0.7980 - f1_score: 0.7980 - recall: 0.7980 - precision: 0.7980
Epoch 5/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4594 - accuracy: 0.7981 - f1_score: 0.7981 - recall: 0.7981 - precision: 0.7981
Epoch 6/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4583 - accuracy: 0.7988 - f1_score: 0.7988 - recall: 0.7988 - precision: 0.7988
Epoch 7/100
5459/5459 [==============================] - 2s 275us/step

In [28]:
best_params_with_layer = random_search.best_params_
valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []
# layers
model = keras.Sequential()
model.add(Dense(best_params_with_layer['num_nodes'][0], input_dim=90, activation='relu'))
for _ in range(best_params_with_layer['num_layers'][0]):
    model.add(Dense(best_params_with_layer['num_nodes'][0], activation='relu'))
    model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy", f1_score, recall, precision])
skf = KFold(n_splits=5, shuffle=True, random_state=30)
for train_index, val_index in skf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    Y_train_fold, Y_val_fold = y_train[train_index], y_train[val_index]
    # Train the model
    model.fit(X_train_fold, Y_train_fold, 
              batch_size=10000, 
              epochs=100, 
              verbose=1, 
              callbacks=[EarlyStopping(min_delta=0.0001, monitor="loss")]
              )
    # Evaluate the model on validation data
    valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_val_fold, Y_val_fold)
    valid_accs.append(valid_acc)
    valid_f1s.append(valid_f1)
    valid_recalls.append(valid_recall)
    valid_precisions.append(valid_precision)
    print("===================================")
    print("Validation accuracy:", valid_acc)
    print("Validation F1-score:", valid_f1)
    print("Validation recall:", valid_recall)
    print("Validation precision:", valid_precision)
print("###################################")
print("avg Validation accuracy:", np.mean(valid_accs))
print("avg Validation F1-score:", np.mean(valid_f1s))
print("avg Validation recall:", np.mean(valid_recalls))
print("avg Validation precision:", np.mean(valid_precisions))


Epoch 1/100
70/70 [==============================] - 1s 6ms/step - loss: 0.5118 - accuracy: 0.7840 - f1_score: 0.7840 - recall: 0.7840 - precision: 0.7840
Epoch 2/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4712 - accuracy: 0.7964 - f1_score: 0.7964 - recall: 0.7964 - precision: 0.7964
Epoch 3/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4656 - accuracy: 0.7970 - f1_score: 0.7970 - recall: 0.7970 - precision: 0.7970
Epoch 4/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4626 - accuracy: 0.7981 - f1_score: 0.7981 - recall: 0.7981 - precision: 0.7981
Epoch 5/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4605 - accuracy: 0.7983 - f1_score: 0.7983 - recall: 0.7983 - precision: 0.7983
Epoch 6/100
70/70 [==============================] - 0s 7ms/step - loss: 0.4593 - accuracy: 0.7987 - f1_score: 0.7987 - recall: 0.7987 - precision: 0.7987
Epoch 7/100
70/70 [==============================] - 0s 7ms/step - los